# Setup

### additional dependencies/configuration

In [1]:
!pip install geopandas --user -q

### modules

In [2]:
# import pyproj
# import geopyspark as gps
# import geopandas as gpd
# import fiona

# from functools import partial
# from pyspark import SparkContext
# from geopyspark.geotrellis.color import ColorMap, get_colors_from_matplotlib
# from geopyspark.geotrellis.tms import TMS
from geonotebook.wrappers import TMSRasterData
from shapely.geometry import MultiPolygon, Polygon, shape
# from shapely.ops import transform

In [3]:
import geopyspark as gps
from pyspark import SparkContext


from overlay.data import data, mask
from overlay import analysis as an

### config spark

In [4]:
conf=gps.geopyspark_conf(appName="ClimateOverlay")
conf.set('spark.master.memory', '8G')
conf.set('spark.ui.enabled', True)
sc = SparkContext(conf = conf)

### create overlay context

In [5]:
hou = an.Overlay('houston')

In [6]:
hou.set_study_area('/home/hadoop/notebooks/data/read/boundaries/houston.shp')

In [7]:
M.set_center(hou.study_area['centroid']['x'], hou.study_area['centroid']['y'], 9)

In [8]:
hou.study_area['base_raster'].construct_map_layer()

In [9]:
M.add_layer(hou.study_area['base_raster'].mappable_layer)

### sea level rise

In [10]:
# download and unzip sea level rise polygons
u = 'https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/TX/TX_HGX_slr_data_dist.zip'
d = "/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/"
# data.zip_to_dir(u, d)

In [11]:
# gdb
gdb = "/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb"

In [12]:
# replace with unquoted filepath to gdb in order to see layers
!ogrinfo /home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb/

Had to open data source read-only.
INFO: Open of `/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb/'
      using driver `OpenFileGDB' successful.
1: TX_HGX_low_0ft (Multi Polygon)
2: TX_HGX_low_1ft (Multi Polygon)
3: TX_HGX_low_2ft (Multi Polygon)
4: TX_HGX_low_3ft (Multi Polygon)
5: TX_HGX_low_4ft (Multi Polygon)
6: TX_HGX_low_5ft (Multi Polygon)
7: TX_HGX_low_6ft (Multi Polygon)
8: TX_HGX_slr_0ft (Multi Polygon)
9: TX_HGX_slr_1ft (Multi Polygon)
10: TX_HGX_slr_2ft (Multi Polygon)
11: TX_HGX_slr_3ft (Multi Polygon)
12: TX_HGX_slr_4ft (Multi Polygon)
13: TX_HGX_slr_5ft (Multi Polygon)
14: TX_HGX_slr_6ft (Multi Polygon)


In [13]:
hou.layersets['sea_level'] = an.LayerSet(an.build_tiled_from_multi_slr(gdb, hou))

In [13]:
M.add_layer(hou.layersets['sea_level'].mappable_layer)

In [14]:
for l in M.layers:
    M.remove_layer(l)

### storm surge

In [37]:
# clip raster
fulltiff = '/home/hadoop/notebooks/data/read/overlay-layers/storm-surge/US_Category5_MOM_Inundation_HighTide.tif'
maskedtiff = '/home/hadoop/notebooks/data/write/overlay-layers/storm-surge/houston_flood_category5.tif'
# mask.maskRasterFromBbox(fulltiff, hou.study_area['bbox'], maskedtiff)

In [38]:
# construct a layerset from a tiff
hou.build_layerset('tiff', 'storm_surge', tiff = maskedtiff)

In [39]:
hou.layersets['storm_surge'].tiled = hou.layersets['storm_surge'].tiled.convert_data_type(gps.CellType.UINT8, 0).repartition(256) 

In [56]:
hou.layersets['storm_surge'].construct_map_layer()

In [57]:
# storm surge layer to the map
M.add_layer(hou.layersets['storm_surge'].mappable_layer)

### flood

In [10]:
# output_shp = '/home/hadoop/notebooks/data/write/overlay-layers/flood/houston_storm_surge.shp'
input_shp = '/home/hadoop/notebooks/data/read/overlay-layers/flood/Fld_Haz_ar_webmer.shp'

In [11]:
clipped_shp = mask.clip_vector_to_boundaries(input_shp, hou)

In [13]:
clipped_shp

'/home/hadoop/notebooks/data/write/overlay-layers/flood/Fld_Haz_ar_webmer_houston.shp'

In [13]:
# import geopandas as gpd
ct = data.read_from_shp(clipped_shp)

In [14]:
hou.build_layerset('polygon', 'flood', polygons = ct, pixel_value = 1)

In [ ]:
M.add_layer(hou.layersets['flood'].mappable_layer)

### Union experiment (remove)

functions

In [16]:
def get_cm(breaks, colors=[0xD28170FF, 0x526095FF]):
    return gps.ColorMap.build(breaks=breaks, colors=[0xD28170FF, 0x526095FF])


def get_mappable(pyr, cm):
    return gps.TMS.build(source=pyr, display=cm)


def mapp(M, pyr, cm):
    for l in M.layers:
        M.remove_layer(l)
    mp = get_mappable(pyr, cm)
    M.add_layer(TMSRasterData(mp))

rasterize_options = gps.RasterizerOptions(includePartial=True, 
                                          sampleType='PixelIsArea')

##### boundary

In [24]:
bounding = hou.study_area['geom']

In [25]:
# rasterize bounding
bd = gps.rasterize(geoms=bounding,
                   crs="EPSG:3857",
                   zoom=12,
                   fill_value=1,
                   cell_type=gps.CellType.FLOAT32,
                   options=rasterize_options,
                   num_partitions=50)

In [26]:
# reclassify bounding
bd_r = bd.reclassify(value_map={1:1}, data_type=int, replace_nodata_with=1)

In [27]:
# pyramid bounding
bd_p = bd_r.pyramid()

In [28]:
# histogram bounding
bd_h = bd_p.get_histogram()

In [29]:
# bin counts
bd_h.bin_counts()

[(1.0, 26083328)]

In [30]:
color_map = get_cm([2, 1])
mapp(M, bd_p, color_map)

##### sea level rise

In [14]:
sea_level_1 = data.read_from_gdb(gdb, 'TX_HGX_slr_1ft', MultiPolygon)

In [17]:
# rasterize bounding
sl = gps.rasterize(geoms=sea_level_1,
                   crs="EPSG:3857",
                   zoom=12,
                   fill_value=2,
                   cell_type=gps.CellType.FLOAT32,
                   options=rasterize_options,
                   num_partitions=50)

In [18]:
# reclassify bounding (from jake)
# sl_r = sl.srdd.reclassify({2:2}, gps.geotrellis.constants.ClassificationStrategy.EXACT.value, 0)
# sl_r = gps.TiledRasterLayer(srdd=sl_r, layer_type=sl.layer_type)

In [19]:
sl_r = sl.reclassify(value_map={2:2}, data_type=int, replace_nodata_with=1)

In [20]:
# pyramid bounding
sl_p = sl_r.pyramid()
# histogram bounding
sl_h = sl_p.get_histogram()
# bin counts
sl_h.bin_counts()

[(1.0, 4501018), (2.0, 33638296)]

In [21]:
color_map = get_cm(breaks=[1, 2])
mapp(M, sl_p, color_map)

##### union

In [22]:
u = gps.geotrellis.union(layers=[bd_r, sl_r])
u_h = u.get_histogram()

NameError: name 'bd_r' is not defined

In [23]:
u_h.bin_counts()

NameError: name 'u_h' is not defined

In [31]:
u_p = u.pyramid()
color_map = get_cm([2, 1])
mapp(M, u_p, color_map)

##### aggregate

In [32]:
ag = u.aggregate_by_cell(operation=gps.geotrellis.constants.Operation.MAX)
ag_h = ag.get_histogram()
ag_h.bin_counts()

[(1.0, 4501018), (2.0, 105990040)]

In [33]:
ag_p = ag.pyramid()
color_map = get_cm(breaks=[2, 1])
mapp(M, ag_p, color_map)

##### Mask

In [34]:
agm = ag.mask(list(hou.study_area['geom']))
agm_h = agm.get_histogram()
agm_h.bin_counts()

[(1.0, 2872777), (2.0, 77681336)]

In [35]:
agm_p = agm.pyramid()
mapp(M, agm_p, get_cm([2, 1]))

##### storm surge

In [40]:
ss = hou.layersets['storm_surge'].tiled
# reclassify bounding
ss_r = ss.reclassify(value_map={100: 11, 90: 10, 80: 9, 70: 8, 60: 7, 50: 6, 40: 5, 30: 4, 20: 3, 10: 2},
                     data_type=int, replace_nodata_with=1)

In [41]:
ss_h = ss_r.get_histogram()
ss_cm = gps.ColorMap.build(breaks=ss_h, colors='GnBu')
ss_p = ss_r.pyramid()
mapp(M, ss_p, ss_cm)

In [42]:
ss_r = ss_r.tile_to_layout(layout=bd_r.layer_metadata)
# I think this is causing a problem
uss = gps.geotrellis.union(layers=[bd_r, ss_r])
uss_h = uss.get_histogram()
uss_h.bin_counts()

[(1.0, 2351602),
 (2.0, 57047943),
 (3.0, 3072771),
 (4.0, 3059034),
 (11.0, 3966428)]

In [43]:
uss_p = uss.pyramid()
uss_cm = gps.ColorMap.build(breaks=uss_h, colors='GnBu')
mapp(M, uss_p, uss_cm)

In [44]:
uss_ag = uss.aggregate_by_cell(
    operation=gps.geotrellis.constants.Operation.MAX)
uss_ag_h = uss_ag.get_histogram()
uss_ag_h.bin_counts()
uss_ag_p = uss_ag.pyramid()
uss_ag_cm = gps.ColorMap.build(breaks=uss_ag_h, colors='GnBu')
mapp(M, uss_ag_p, uss_ag_cm)

In [45]:
uss_agm = uss_ag.mask(list(hou.study_area['geom']))
uss_agm_h = uss_agm.get_histogram()
uss_agm_h.bin_counts()
uss_agm_p = uss_agm.pyramid()
uss_agm_cm = gps.ColorMap.build(breaks=uss_agm_h, colors='GnBu')
mapp(M, uss_agm_p, uss_agm_cm)

In [46]:
uss_agm_h.bin_counts()

[(1.0, 1360988),
 (2.0, 13354231),
 (3.0, 1885668),
 (4.0, 1983779),
 (11.0, 2885740)]

Test

In [47]:
test = uss_agm + agm

In [48]:
test_h = test.get_histogram()

In [49]:
test_h.bin_counts()

[(2.0, 3986881),
 (3.0, 11692302),
 (4.0, 3270441),
 (5.0, 1363387),
 (6.0, 517324),
 (12.0, 86124),
 (13.0, 279398)]

In [50]:
test_pyr = test.pyramid()

In [51]:
test_cm = gps.ColorMap.build(breaks=test_h, colors='GnBu')

In [52]:
mapp(M, test_pyr, test_cm)